# 종목코드 데이터 불러오기

In [1]:
import pandas as pd

In [2]:
df = pd.read_html('https://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0]
df_stock = df.loc[:,['종목코드', '회사명']]
df_stock.loc[:,'종목코드'] = df_stock.loc[:,'종목코드'].apply(str).str.rjust(6, '0')

In [3]:
df

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DL,210,기타 금융업,지주회사,1976-02-02,12월,전병욱,http://www.dlholdings.co.kr,서울특별시
1,DRB동일,4840,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매",1976-05-21,12월,류영식,http://drbworld.com,부산광역시
2,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
3,GS,78930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
4,KG케미칼,1390,기초 화학물질 제조업,"콘크리트혼화제, 비료, 친환경농자재, 수처리제",1989-08-25,12월,"곽정현, 김재익",http://www.kgchem.co.kr,울산광역시
...,...,...,...,...,...,...,...,...,...
2499,코스텍시스템,169670,특수 목적용 기계 제조업,"반도체 웨이퍼 이송장비, 본딩장비",2022-01-21,12월,배준호,http://www.kosteks.com/,경기도
2500,타임기술,318660,소프트웨어 개발 및 공급업,"ILS(종합군수지원), IETM(전자식기술교범), CBT(전자식교보재), 기술번역",2021-06-24,12월,주양효,http://timett.co.kr,경상남도
2501,플럼라인생명과학,222670,자연과학 및 공학 연구개발업,"강아지암치료제,돼지DNA테라피",2015-07-28,12월,김앤토니경태,http://www.plsdna.com,서울특별시
2502,한국미라클피플사,331660,기타 화학제품 제조업,"은나노스텝, 닥터오렌지 등",2019-10-28,12월,이호경,http://www.kmpc.co.kr,경기도


In [4]:
df_stock.head(10)

,종목코드,회사명
0,000210,DL
1,004840,DRB동일
2,155660,DSR
3,078930,GS
4,001390,KG케미칼
5,025000,KPX케미칼
6,058860,KTis
7,011070,LG이노텍
8,037560,LG헬로비전
9,108670,LX하우시스


In [5]:
stocks = list(df_stock['종목코드'])

In [6]:
stocks[:10]

['000210',
 '004840',
 '155660',
 '078930',
 '001390',
 '025000',
 '058860',
 '011070',
 '037560',
 '108670']

# 주식 데이터 크롤링

In [7]:
import requests
from bs4 import BeautifulSoup as BS
import re
from datetime import time, datetime

In [8]:
head = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36"}

In [9]:
def crawl_time_data(code):
    thistime = datetime.now().strftime('%Y%m%d%H%M%S')
    url = 'https://finance.naver.com/item/sise_time.naver?code='+code+'&thistime='+thistime+''
    r = requests.get(url, headers=head)
    bs = BS(r.text, 'lxml')
    
    stock = []
    data = bs.find("table", class_="type2").findAll("tr")[2]
    td = data.findAll("td")
    stock.append([y.text.strip() for y in td])
    print(stock)

In [10]:
crawl_time_data('005930')

[['09:13', '58,600', '200', '58,600', '58,500', '2,780,111', '237']]


In [11]:
import pandas as pd
from datetime import time, datetime
import requests 

In [30]:
import warnings
warnings.filterwarnings(action='ignore')

In [37]:
def crawl_time_data_all(code):
    thistime = datetime.now().strftime('%Y%m%d%H%M%S')
    df=pd.DataFrame()
    for page in range(1, 42):
        url = 'https://finance.naver.com/item/sise_time.naver?code='+code+'&thistime='+thistime+''
        pg_url = '{url}&page={page}'.format(url=url, page=page)
        print(pg_url)
        df = df.append(pd.read_html(requests.get(pg_url, headers=head).text, header=0)[0], ignore_index=True)
    return df 

In [39]:
df2 = crawl_time_data_all('005930')

https://finance.naver.com/item/sise_time.naver?code=005930&thistime=20220627102915&page=1
https://finance.naver.com/item/sise_time.naver?code=005930&thistime=20220627102915&page=2
https://finance.naver.com/item/sise_time.naver?code=005930&thistime=20220627102915&page=3
https://finance.naver.com/item/sise_time.naver?code=005930&thistime=20220627102915&page=4
https://finance.naver.com/item/sise_time.naver?code=005930&thistime=20220627102915&page=5
https://finance.naver.com/item/sise_time.naver?code=005930&thistime=20220627102915&page=6
https://finance.naver.com/item/sise_time.naver?code=005930&thistime=20220627102915&page=7
https://finance.naver.com/item/sise_time.naver?code=005930&thistime=20220627102915&page=8
https://finance.naver.com/item/sise_time.naver?code=005930&thistime=20220627102915&page=9
https://finance.naver.com/item/sise_time.naver?code=005930&thistime=20220627102915&page=10
https://finance.naver.com/item/sise_time.naver?code=005930&thistime=20220627102915&page=11
https://

In [40]:
df2

,체결시각,체결가,전일비,매도,매수,거래량,변동량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10:29,59200.0,800.0,59200.0,59100.0,6837349.0,1.0
2,10:28,59100.0,700.0,59200.0,59100.0,6837348.0,42505.0
3,10:27,59100.0,700.0,59200.0,59100.0,6794843.0,38534.0
4,10:26,59100.0,700.0,59100.0,59000.0,6756309.0,50223.0
...,...,...,...,...,...,...,...
610,09:03,58700.0,300.0,58700.0,58600.0,1404189.0,178576.0
611,09:02,58800.0,400.0,58800.0,58700.0,1225613.0,196764.0
612,09:01,58900.0,500.0,58900.0,58800.0,1028849.0,243050.0
613,09:00,58700.0,300.0,58800.0,58700.0,785799.0,785799.0
